In [39]:
#Parameters
N=20; #songs in the playlist

In [53]:
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd
import random

In [116]:
#Load data
dataSet = pd.read_pickle('data/dataSet.csv')
coords2D = np.load("data/coords2D.npy").item()
dfmax=np.load('data/dfmax.npy')
tracks = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1]);
tracks_raw = pd.read_csv('../data/fma_metadata/raw_tracks.csv', index_col=0);

In [4]:
# Convert to np.array
points_ref = np.array([tuple(x) for x in dataSet.drop(columns='label',level=0).values])

# Construct the KD Tree
tree = cKDTree(points_ref)

In [40]:
# Loading custom songs features
features_c = pd.read_csv('data/custom_features.csv', index_col=0, header=[0, 1, 2], sep=',', encoding='utf-8')
tid_c = pd.read_csv('data/custom_tid.csv', index_col=0, header=[0], sep=',', encoding='utf-8')

features_c=features_c.divide(dfmax)
numb_cust=len(features_c)
features_c

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
-1            0.000040 -0.000210 -0.000410 -0.000130  0.000480  0.000845   
-2            0.000008  0.000342  0.000633  0.000173 -0.000063  0.000007   
-3            0.000045 -0.000121  0.000470 -0.000084 -0.000264 -0.000272   
-4           -0.000090 -0.000168 -0.000703 -0.000249 -0.000190 -0.000294   
-5            0.000801  0.001783  0.000376  0.000315  0.000443  0.000175   
-6            0.000034 -0.000189  0.000169  0.000012 -0.000119 -0.000222   
-7           -0.000014 -0.000086 -0.000243 -0.000153 -0.000054 -0.000049   
-8            0.000801  0.001783  0.000376  0.000315  0.000443  0.000175   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
track_id                                              ...                
-1          0.000117 -0.000020  0.000193 -0.000021    ...     0.283257   
-2         -0.000102 -0.000063  0.000060 -0.000251    ...     0.277270   
-3          0.000211  0.000181  0.000106 -0.000429    ...     0.350125   
-4         -0.000254  0.000211  0.000168 -0.000311    ...     0.391784   
-5          0.000327  0.000692  0.000757  0.000758    ...     0.168253   
-6          0.000022  0.000011  0.000394 -0.000348    ...     0.279832   
-7         -0.000109  0.000486  0.000226  0.000035    ...     0.244431   
-8          0.000327  0.000692  0.000757  0.000758    ...     0.168253   

feature                              zcr                                \
statistics                      kurtosis       max      mean    median   
number            05        06        01        01        01        01   
track_id                                                                 
-1          0.201160  0.127691  0.000817  0.393258  0.083793  0.074114   
-2          0.222513  0.155552  0.000762  0.410357  0.099193  0.093188   
-3          0.251780  0.175684  0.008211  0.616512  0.049449  0.040327   
-4          0.238435  0.219134  0.006972  0.660479  0.043547  0.028883   
-5          0.135252  0.101407  0.000683  0.459697  0.097202  0.079564   
-6          0.172038  0.176377  0.008916  0.340010  0.028089  0.021798   
-7          0.156275  0.148947  0.001793  0.567660  0.080811  0.073569   
-8          0.135252  0.101407  0.000683  0.459697  0.097202  0.079564   

feature                                   
statistics       min      skew       std  
number            01        01        01  
track_id                                  
-1          0.000000  0.025611  0.110382  
-2          0.000000  0.018219  0.084653  
-3          0.013035  0.075646  0.083336  
-4          0.018622  0.091880  0.143848  
-5          0.000000  0.026778  0.144958  
-6          0.000000  0.091840  0.057087  
-7          0.003724  0.030420  0.093448  
-8          0.000000  0.026778  0.144958  

[8 rows x 518 columns]

In [6]:
# Convert to np.array
features_c_t = np.array([tuple(x) for x in features_c.values])

# Find the 10th nearest points with Euclidean norm in the tree
dist, idx = tree.query(features_c_t, k=N, p=2)

# Playlist 518D

In [13]:
track_r = tracks.reset_index()
#list of the tracks nearest to each custom song
nearest_tracks=list()

# For every added song we fetch from the query
for tid, nodes in enumerate(idx):
    
    # Retrieve the custom id and display the name
    id = features_c.iloc[tid].name
    print("Track id:", id)
    print("Track name:", tid_c[tid_c.index == id].values[0][0])

    # From the neighbours nodes list, retrieve their track id, artist and title
    track_list = list(dataSet.iloc[list(nodes)].index.values)
    results = track_r[track_r['track_id'].isin(track_list)]
    
    # Create a DataFrame to summarize
    r_df = pd.DataFrame({'track_id': track_list, \
                         'artist': results['artist']['name'].values, \
                         'title': results['track']['title'].values, \
                         'distance': dist[tid]})
    nearest_tracks.append(track_list[0])
   
    display(r_df)
    print('\n')

Track id: -1
Track name: The Hit - Tupper Ware Remix Party (feat. Ninja Sex Party)


,track_id,artist,title,distance
0,64989,6th Sense &amp; Mick Boogie,Belief feat. John Mayer,1.270235
1,11839,Katrah-Quey,Crown City Rockers - No Sense (Remix),1.310617
2,120518,Kreamy 'Lectric Santa,Me so horny 2,1.338916
3,89456,Alaclair Ensemble,Fa' faire du loot,1.356200
4,32882,Kellee Maize,Hasta Abajo,1.356911
5,96401,Action Davis,Bee Mine,1.370840
6,68601,The Hate Noise,Dropout Boogie,1.372639
7,75221,Holy Pain,The Ambassador Of Death,1.386899
8,55121,Alaclair Ensemble,Montagnes russes (Serge Fiori),1.387280
9,96399,Valley Lodge,Every Little Thing,1.392382




Track id: -2
Track name: 13 On Mercury


,track_id,artist,title,distance
0,14693,Jonathan Coulton,I Feel Fantastic,1.280966
1,95506,Single Bullet Theory,Miss Two Knives,1.283501
2,131656,The Pleasure Kills,Oblivion,1.292824
3,99345,Sick of Sarah,Kick Back,1.308903
4,95910,Oddish,Engano Seu,1.319336
5,114199,Square People,I'm Not Lazy,1.320697
6,112468,Mudhoney,Slipping Away,1.321080
7,33992,Flamin' Groovies,Shake Some Action,1.352147
8,133731,Suicidal Rap Orgy,Murder Rap,1.361353
9,84290,Still Pluto,Bitter Pill (feat Vosmoy),1.364139




Track id: -3
Track name: 02. Sorceress


...

,track_id,artist,title,distance
0,129695,Curtis Eller,Black Diamond,1.386336
1,125194,Koonda Holaa,It's Not Like You (Always),1.443722
2,49845,Koonda Holaa,Gadget,1.604237
3,97570,Calexico,Roka,1.609812
4,85489,My Bubba & Mi,Steamengeene,1.611600
5,12350,The Jacobins,Hazy Days,1.647572
6,126743,Derek Clegg,Easy,1.663117
7,33538,E-Prosounds,Eeze Out / Wherever the Wind Blow,1.703908
8,12351,Zach Fleury,"Hey, Marie.",1.713455
9,108014,A Crooked Pulse,Once the Sky,1.715689




Track id: -7
Track name: 01 Out of the Black


,track_id,artist,title,distance
0,114200,Black Elk,Toggle,1.140494
1,48369,Half Japanese,Thing With a Hook,1.150064
2,126667,Lorenzo's Music,Feed The One,1.174115
3,123278,Cloudkicker,One Enemy Among Twenty-Five Million Friends,1.183574
4,4782,Mutilation Rites,A Season of Grey Rain,1.199688
5,35534,Starring,The Best,1.223132
6,256,Brutal Whiskers,Узник,1.226216
7,89473,Angstbreaker,Dead Elements,1.231278
8,83791,Public Enemy,Get Up Stand Up (Featuring Brother Ali),1.234839
9,64567,Acherontia,Human,1.239352




Track id: -8
Track name: glaaa


,track_id,artist,title,distance
0,2,AWOL,Food,0.340773
1,6610,Height With Friends,Clack Struck Twelve,1.474426
2,135369,BrokeMC,The Discovery,1.499477
3,7492,BrokeMC,In the Dark,1.510797
4,7488,Opio,Different,1.515337
5,57500,Barreracudas,Girl,1.524906
6,108031,Josh Woodward,Stickybee,1.534824
7,108500,Hi-Rez,The Remedy (Prod. Joel Johnston),1.543426
8,98584,Hi-Rez,Relax (Prod. Joel Johnston),1.546447
9,117883,Wann,Happy Birthday,1.559435


In [180]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, 'Open')+'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp'+'<a target="_blank" href="{}">{}</a>'.format(val+'/download', 'Download')



In [186]:
#find disc
graphCoords=list()
for i in coords2D:
    graphCoords.append(tuple(coords2D[i]))
graphCoords=np.array(graphCoords)
graphTree = cKDTree(graphCoords)

playlist=list()

for i in nearest_tracks:
    dist, idx = graphTree.query(tuple(coords2D[i]), k=np.ceil(N/numb_cust), p=2)
    playlist.append([list(coords2D.keys())[i] for i in idx])

playlist=[y for x in playlist for y in x]
playlist=random.sample(playlist,N)

tracks.filter(playlist, axis=0)
result=pd.DataFrame({'Artist': tracks.artist.name[playlist],'Title':tracks.track.title[playlist],'Links':tracks_raw['track_url'][playlist]}) 
#                          'artist': results['artist']['name'].values, \
#                          'title': results['track']['title'].values})
# nearest_tracks.append(track_list[0])
# tracks[[]].isin(playlist)
result.reset_index(inplace=True)
result.drop(columns='track_id',inplace=True)
result.style.format({'Links':make_clickable})


,Artist,Title,Links
0,Derek Clegg,All of My Love Resides in Paris,Open Download
1,Jonathan Coulton,I Feel Fantastic,Open Download
2,Action Davis,Bee Mine,Open Download
3,Upsilon Acrux,So Thereby...and furthermore...thus henceforth...Mono,Open Download
4,Kellee Maize,Friday Night Flu (Remix),Open Download
5,godmanwho,10 LIKE LOATHE OR LOVE IT,Open Download
6,Aviscerall,Whatever You Want (Feat. Crystal Tokyo),Open Download
7,Kristin Hersh,Moan,Open Download
8,AWOL,Food,Open Download
9,Dälek,Classical Homicide,Open Download
